## HW1 13scene

### Import Packages

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import os 
%matplotlib inline
np.random.seed(2)

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools

# import keras
import keras
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Convolution2D, MaxPool2D, BatchNormalization, SeparableConv2D, Activation
from keras.optimizers import RMSprop, SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau, LearningRateScheduler, ModelCheckpoint
from keras.layers import Input
from tensorflow.python.keras.models import Model
from keras.preprocessing import image
from keras.models import load_model
sns.set(style='white', context='notebook', palette='deep')

# import PIL
from PIL import Image
import glob

# import scipy
from scipy.signal import convolve2d

Using TensorFlow backend.


In [2]:
num_class = 13
input_size = 256
batch_size = 64

### Define VGG19 model

In [4]:
from keras.applications import VGG19
def vgg19_model():
    model = Sequential()
    model.add(VGG19(include_top=False, weights='imagenet', classes=num_class, input_shape=(256,256,3)))
    model.add(Flatten())
    model.add(Dense(256))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.4))
    model.add(Dense(num_class, activation='softmax'))
    #model.layers[0].trainable = False
    return model
    pass
classifier = vgg19_model()
classifier.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Model)                (None, 8, 8, 512)         20024384  
_________________________________________________________________
flatten_1 (Flatten)          (None, 32768)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               8388864   
_________________________________________________________________
batch_normalization_1 (Batch (None, 256)               1024      
_________________________________________________________________
activation_1 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 13)               

#### show Model Structure

In [5]:
for i, layer in enumerate(classifier.layers):
    if i == 0:
        for layer0 in classifier.layers[i].layers[:]:
            layer0.trainable = False
            print('-'*40)
            print('%-25s'%layer0.name, '%10s'%layer0.trainable)
    else:
        print('-'*40)
        print('%-25s'%layer.name, '%10s'%layer.trainable)

----------------------------------------
input_1                        False
----------------------------------------
block1_conv1                   False
----------------------------------------
block1_conv2                   False
----------------------------------------
block1_pool                    False
----------------------------------------
block2_conv1                   False
----------------------------------------
block2_conv2                   False
----------------------------------------
block2_pool                    False
----------------------------------------
block3_conv1                   False
----------------------------------------
block3_conv2                   False
----------------------------------------
block3_conv3                   False
----------------------------------------
block3_conv4                   False
----------------------------------------
block3_pool                    False
----------------------------------------
block4_conv1           

### Import 13-scene Dataset & Preprocessing

In [6]:
def training_size(full_data=False, validation_split_ratio=0.2):
    if full_data:
        valid_split = 0.05
    else:
        valid_split = validation_split_ratio
        
    train_datagen = ImageDataGenerator(
        height_shift_range=0.2,
        width_shift_range=0.2,
        zoom_range=0.2,
        shear_range=0.2,
        validation_split=valid_split)

    valid_datagen = ImageDataGenerator(
        validation_split=valid_split)
    
    return train_datagen, valid_datagen
train_datagen, valid_datagen = training_size(full_data=True, validation_split_ratio=0.2)

In [8]:
training_set = train_datagen.flow_from_directory(
    'scene/data',
    target_size=(256, 256),
    subset='training',
    batch_size=batch_size,
    class_mode='categorical',
    seed=3023)

validation_set = valid_datagen.flow_from_directory(
    'scene/data',
    target_size=(256, 256),
    subset='validation',
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False,
    seed=3023)

Found 3672 images belonging to 13 classes.
Found 187 images belonging to 13 classes.


### Build Model

In [10]:
# optimizer = RMSprop(lr=0.0001, rho=0.9, epsilon=1e-08, decay=0.0)
# optimizer = SGD(lr=0.0001, momentum=0.9)

#classifier = vgg19_model()
classifier.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

#### save model when meets the highest validation set accuracy

In [11]:
ac = ModelCheckpoint('pretrain_model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)

In [12]:
classifier.fit_generator(
    training_set,
    steps_per_epoch=58,
    epochs=30,
    validation_data=validation_set,
    validation_steps=32.5,
    verbose=1,
    callbacks = [ac]
)

Epoch 1/30
58/58 [==============================] - 90s 2s/step - loss: 0.7266 - accuracy: 0.7590 - val_loss: 0.7863 - val_accuracy: 0.8663

Epoch 00001: val_accuracy improved from -inf to 0.86631, saving model to 13SCENE_bs64_spe58.h5
Epoch 2/30
58/58 [==============================] - 75s 1s/step - loss: 0.3271 - accuracy: 0.8903 - val_loss: 0.1846 - val_accuracy: 0.9198

Epoch 00002: val_accuracy improved from 0.86631 to 0.91979, saving model to 13SCENE_bs64_spe58.h5
Epoch 3/30
58/58 [==============================] - 77s 1s/step - loss: 0.2525 - accuracy: 0.9156 - val_loss: 0.2355 - val_accuracy: 0.9412

Epoch 00003: val_accuracy improved from 0.91979 to 0.94118, saving model to 13SCENE_bs64_spe58.h5
Epoch 4/30
58/58 [==============================] - 76s 1s/step - loss: 0.2251 - accuracy: 0.9243 - val_loss: 0.2506 - val_accuracy: 0.9144

Epoch 00004: val_accuracy did not improve from 0.94118
Epoch 5/30
58/58 [==============================] - 77s 1s/step - loss: 0.1958 - accuracy: